# Temperature change in Europe over next 48H Dashboard

# 1 Intro

## 1.1 Motivation for Development

受IOS 天气app 启发. IOS的dashboard只能播放降水的未来变化, 我想做一个关于气温的. 这样人们就可以很清楚冷/热空气团的移动, 对于要在短时间内, 在欧洲多地周转的人们很有帮助

# 2 Library files to use

In [10]:
import requests
import pandas as pd
import json
import csv

# 3 Weather data acquisition

## 3.1 Weather API Testing

这里我们使用 "Weather API" 进行测试, 首先申请到API_KEY, 然后进行测试API接口是否正常工作

In [4]:
WEATHER_API_KEY = "7aa473fe613f4cef864171058242804"
WEATHER_API_URL = "http://api.weatherapi.com/v1/forecast.json"

In [5]:
# test the API with london
response_allCity = requests.get(WEATHER_API_URL, params={"key": WEATHER_API_KEY, "q": "London", "days": 1})
print(response_allCity.status_code)
print(response_allCity.json())

200
{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.52, 'lon': -0.11, 'tz_id': 'Europe/London', 'localtime_epoch': 1714333552, 'localtime': '2024-04-28 20:45'}, 'current': {'last_updated_epoch': 1714332600, 'last_updated': '2024-04-28 20:30', 'temp_c': 10.0, 'temp_f': 50.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 220, 'wind_dir': 'SW', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 66, 'cloud': 25, 'feelslike_c': 8.7, 'feelslike_f': 47.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 3.0, 'gust_mph': 11.7, 'gust_kph': 18.9}, 'forecast': {'forecastday': [{'date': '2024-04-28', 'date_epoch': 1714262400, 'day': {'maxtemp_c': 9.1, 'maxtemp_f': 48.3, 'mintemp_c': 5.0, 'mintemp_f': 41.1, 'avgtemp_c': 7.4, 'avgtemp_f': 45.2, 'maxwind_mph': 13.6, 'maxwin

## 3.2 Get temperature data for a single city

### 3.2.1 Encapsulate apis as a function

根据返回的结果, 我们可以看出该接口正常工作

接下来, 由于我们要获得欧洲大部分城市的气温数据, 所以我们要定义一个函数, 来方便调用

In [22]:
def get_weather_data(city, days = 2):
    params = {
        "key": WEATHER_API_KEY,
        "q": city,
        "days": days
    }
    response = requests.get(WEATHER_API_URL, params=params)
    formatted_response = json.dumps(response.json(), indent=4) # pretty print
    return formatted_response

In [23]:
# test the function
response_allCity = get_weather_data("London")
print(response_allCity)

{
    "location": {
        "name": "London",
        "region": "City of London, Greater London",
        "country": "United Kingdom",
        "lat": 51.52,
        "lon": -0.11,
        "tz_id": "Europe/London",
        "localtime_epoch": 1714334159,
        "localtime": "2024-04-28 20:55"
    },
    "current": {
        "last_updated_epoch": 1714333500,
        "last_updated": "2024-04-28 20:45",
        "temp_c": 10.0,
        "temp_f": 50.0,
        "is_day": 0,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/night/116.png",
            "code": 1003
        },
        "wind_mph": 8.1,
        "wind_kph": 13.0,
        "wind_degree": 220,
        "wind_dir": "SW",
        "pressure_mb": 1011.0,
        "pressure_in": 29.85,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 66,
        "cloud": 25,
        "feelslike_c": 8.7,
        "feelslike_f": 47.6,
        "vis_km": 10.0,
        "vis_mile

### 3.2.2 Extract required data

Extract the required data from the interface return value

In [24]:
data = json.loads(response_allCity) # convert the json string to a dictionary

country = data['location']['country']
name = data['location']['name']
temperatures = [hour['temp_c'] for day in data['forecast']['forecastday'] for hour in day['hour']][:48]

print(f"Country: {country}")
print(f"Name: {name}")
print(f"Temperatures for the next 48 hours: {temperatures}")

Country: United Kingdom
Name: London
Temperatures for the next 48 hours: [7.7, 7.8, 7.9, 7.9, 8.2, 8.5, 8.5, 8.6, 8.1, 5.4, 5.0, 5.3, 5.8, 6.7, 7.2, 8.1, 8.7, 8.9, 9.1, 7.9, 10.0, 6.6, 6.1, 5.8, 5.6, 5.4, 5.2, 5.1, 5.1, 5.2, 6.3, 8.4, 10.5, 12.0, 12.8, 13.4, 14.0, 14.4, 14.4, 14.2, 13.5, 13.1, 12.4, 11.3, 10.1, 9.6, 9.3, 9.1]


### 3.2.3 Save required data to file

Save it in "temperature.csv" in the same folder

In [25]:
def write_temperatures_to_csv(response):
    data = json.loads(response)
    
    country = data['location']['country']
    name = data['location']['name']
    temperatures = [hour['temp_c'] for day in data['forecast']['forecastday'] for hour in day['hour']][:48]
    
    with open('temperature.csv', 'a', newline='') as file:  # 'a' is for append mode
        writer = csv.writer(file)
        writer.writerow([country, name, temperatures])
        
    print(f"Country: {country}, Name: {name}, Temperatures: {temperatures}")

In [26]:
# test write_temperatures_to_csv function
write_temperatures_to_csv(response_allCity)

Country: United Kingdom, Name: London, Temperatures: [7.7, 7.8, 7.9, 7.9, 8.2, 8.5, 8.5, 8.6, 8.1, 5.4, 5.0, 5.3, 5.8, 6.7, 7.2, 8.1, 8.7, 8.9, 9.1, 7.9, 10.0, 6.6, 6.1, 5.8, 5.6, 5.4, 5.2, 5.1, 5.1, 5.2, 6.3, 8.4, 10.5, 12.0, 12.8, 13.4, 14.0, 14.4, 14.4, 14.2, 13.5, 13.1, 12.4, 11.3, 10.1, 9.6, 9.3, 9.1]


## 3.3 Get temperature data from all Europe cities

### 3.3.1 List all the Europe main cities

In [17]:
# list all the European main cities
cities = [
    "London",
    "Paris",
    "Berlin",
    "Madrid",
    "Rome",
    "Lisbon",
    "Brussels",
    "Amsterdam",
    "Vienna",
    "Warsaw",
    "Prague",
    "Budapest",
    "Stockholm",
    "Copenhagen",
    "Oslo",
    "Helsinki",
    "Dublin",
    "Athens",
    "Sofia",
    "Bucharest",
    "Riga",
    "Vilnius",
    "Tallinn",
    "Valletta",
    "Nicosia",
    "Luxembourg",
    "Bratislava",
    "Ljubljana",
    "Zagreb",
    "Nicosia",
    "Andorra la Vella",
    "Monaco",
    "San Marino",
    "Vatican City",
]

In [27]:
# get the weather data for all the cities
for city in cities:
    response_allCity = get_weather_data(city)
    write_temperatures_to_csv(response_allCity)
    
    print(f"Data for {city} has been written to temperature.csv")

Country: United Kingdom, Name: London, Temperatures: [7.7, 7.8, 7.9, 7.9, 8.2, 8.5, 8.5, 8.6, 8.1, 5.4, 5.0, 5.3, 5.8, 6.7, 7.2, 8.1, 8.7, 8.9, 9.1, 7.9, 10.0, 6.6, 6.1, 5.8, 5.6, 5.4, 5.2, 5.1, 5.1, 5.2, 6.3, 8.4, 10.5, 12.0, 12.8, 13.4, 14.0, 14.4, 14.4, 14.2, 13.5, 13.1, 12.4, 11.3, 10.1, 9.6, 9.3, 9.1]
Data for London has been written to temperature.csv
Country: France, Name: Paris, Temperatures: [9.9, 11.0, 9.4, 9.6, 9.2, 9.7, 9.4, 9.7, 9.6, 10.2, 9.6, 11.6, 13.9, 14.2, 15.3, 16.1, 16.5, 15.4, 14.2, 13.0, 11.0, 10.2, 9.7, 9.1, 8.6, 8.3, 8.1, 7.7, 7.5, 7.4, 7.2, 8.7, 10.3, 12.0, 13.7, 15.3, 16.7, 17.7, 18.1, 18.2, 18.1, 18.0, 17.7, 16.2, 14.0, 13.5, 13.2, 13.2]
Data for Paris has been written to temperature.csv
Country: Germany, Name: Berlin, Temperatures: [12.4, 11.8, 11.2, 10.7, 10.4, 10.7, 11.3, 12.8, 14.4, 16.7, 18.4, 20.0, 21.7, 22.8, 23.5, 23.7, 23.7, 23.3, 21.7, 19.5, 17.6, 18.0, 15.8, 15.2, 14.5, 13.9, 13.3, 12.7, 12.6, 12.1, 12.1, 12.6, 13.3, 13.9, 15.7, 16.4, 16.8, 18.6, 